# Chapter 17a - supplement: Advanced composite types

There are some additional information that is helpful to know about types.  We will cover the difference between inner and outer constructors as well as parametric types in more detail.

## 17.X inner and outer constructors
So far, we have only talked about inner constructors.  Let's return to the example of Cards.  The following uses only inner constructors:

In [1]:
struct Card
  rank::Int
  suit::Int

  # construct a card based on the rank and suit
  function Card(r::Int,s::Int)
    1 <= r <=13  || throw(ArgumentError("The rank must be an integer between 1 and 13."))
    1 <= s <= 4  || throw(ArgumentError("The suit must be an integer between 1 and 4."))
    new(r,s)
  end

  # construct a card based on the number in a deck
  function Card(i::Int)
    1 <= i <= 52 || throw(ArgumentError("The argument must be an integer between 1 and 52"))
    i%13==0 ? new(13,div(i,13)) : new(i%13,div(i,13)+1)
  end
end

First, notice that both of the functions have the same name as the struct.  This makes them constructor functions.  Since they are both inside the struct, they are *inner constructors*.  The following would be the same with *outer construtors*

In [12]:
struct Card2
  rank::Int
  suit::Int
end

# construct a card based on the rank and suit
function Card2(r::Int,s::Int)
  1 <= r <=13  || throw(ArgumentError("The rank must be an integer between 1 and 13."))
  1 <= s <= 4  || throw(ArgumentError("The suit must be an integer between 1 and 4."))
  new(r,s)
end

# construct a card based on the number in a deck
function Card2(i::Int)
  1 <= i <= 52 || throw(ArgumentError("The argument must be an integer between 1 and 52"))
  i%13==0 ? Card2(13,div(i,13)) : Card2(i%13,div(i,13)+1)
end

Card2

Notice that for outer constructors you can't use the `new` function, but instead, use the name of the `struct`.

In [13]:
Card2(3,4)

LoadError: UndefVarError: new not defined

In [11]:
Card2(45)

LoadError: StackOverflowError:

And you can mix and match these as well.

In [14]:
struct Card3
  rank::Int
  suit::Int


  # construct a card based on the rank and suit
  function Card3(r::Int,s::Int)
    1 <= r <=13  || throw(ArgumentError("The rank must be an integer between 1 and 13."))
    1 <= s <= 4  || throw(ArgumentError("The suit must be an integer between 1 and 4."))
    new(r,s)
  end
end

# construct a card based on the number in a deck
function Card3(i::Int)
  1 <= i <= 52 || throw(ArgumentError("The argument must be an integer between 1 and 52"))
  i%13==0 ? Card3(13,div(i,13)) : Card3(i%13,div(i,13)+1)
end

Card3

In [15]:
Card3(3,4)

Card3(3, 4)

In [16]:
Card3(45)

Card3(6, 4)

### When to use an inner versus outer constructor?
In these example, it seems like it doesn't matter if your using inner or outer constructors.  However, if you only had the struct in the form:
```
struct Card
  rank::Int
  suit::end
end
```
and the outer constructors were not yet defined, you could make a card with `Card(18,4)`, which doesn't make sense (and when trying to print out nicely like we did in Chapter 12 we get an error.  

You can use the rule of thumb: if you need to do parameter checking before creating the object, use an inner constructor.  Otherwise, you can do either.

### Why not always use inner constructors?

A very good question.  Typically when writing your own types (structs), you have control over the code and can do what ever you like.  However, outer constructors allow you to add functionality to existing types (either built-in or from packages.)

Consider the example of `Rational` data type.  We usually made them in the form `1//2` with the double `/` but can also make them with the `Rational` constructor. 

In [17]:
1//2

1//2

In [18]:
Rational(1,2)

1//2

In [19]:
Rational(3)

3//1

In [20]:
Rational()

LoadError: MethodError: no method matching Rational()
[0mClosest candidates are:
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  (::Type{T})([91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:266
[0m  (::Type{T})([91m::Complex[39m) where T<:Real at complex.jl:44
[0m  ...

Probably the empty constructor should have an error, but let's say we want to make the empty one be the rational number 0

In [21]:
Rational() = Rational(0,1)

Rational

In [22]:
Rational()

0//1

## 17.x2 Parametric Types

We have seen parametric data types before with the `Polynomial` struct from Chapter 12. We now extend this idea.

In [1]:
struct Point2D{T <: Real}
  x::T
  y::T
end

In [2]:
Point2D(1,1)

Point2D{Int64}(1, 1)

In [3]:
Point2D(1.0,2.5)

Point2D{Float64}(1.0, 2.5)

In [4]:
Point2D(11//2,6//4)

Point2D{Rational{Int64}}(11//2, 3//2)

In [5]:
Point2D(1,1.5)

LoadError: MethodError: no method matching Point2D(::Int64, ::Float64)
[0mClosest candidates are:
[0m  Point2D(::T, [91m::T[39m) where T<:Real at In[1]:2

The problem with this struct is that the way we have defined it that both coordinates need to be the same type and once we mix types, then we have a problem. We'll fix this with a constructor.  

Let's switch over to a file now to fix this.  

In [1]:
using Revise

In [2]:
includet("Points.jl")
using .Points

In [3]:
Point2D(1,1)

Point2D{Int64}(1, 1)

In [14]:
Point2D(1,1.5)

Point2D{Float64}(1.0, 1.5)

In [15]:
Point2D(1,3//2)

Point2D{Rational{Int64}}(1//1, 3//2)

In [16]:
Point2D(33//12,1.5)

Point2D{Float64}(2.75, 1.5)

In [7]:
33//12+1.5

4.25